In [3]:
import json
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

seeds = []
for i in range(1, 6):
    seeds.append(i)

def time_to_minutes(timestr):
    # Handle '+1' suffix by removing it before parsing
    timestr = timestr.split('+')[0]  # Remove '+1' if present
    hh, mm = timestr.split(':')
    return int(hh) * 60 + int(mm)



def calculate_slack_for_scenario(scenario_data):
    """
    Calculate the slack metric for the given scenario.
    
    Slack is defined as:
        Slack = 1 - (total flight minutes in recovery period / total recovery period aircraft-minutes)
    
    A slack of 1 means no flights during recovery period.
    A slack of 0 means flights occupy the entire recovery period.
    """
    
    # Extract scenario start/end times
    # We assume the same date for start and end for simplicity.
    recovery_start_time_str = scenario_data["recovery_start_time"]  
    recovery_end_time_str = scenario_data["recovery_end_time"]      
    
    recovery_start_minutes = time_to_minutes(recovery_start_time_str)
    recovery_end_minutes = time_to_minutes(recovery_end_time_str)
    total_recovery_period_minutes = recovery_end_minutes - recovery_start_minutes
    
    total_aircraft = scenario_data["total_aircraft"]
    
    # Calculate total flight minutes within the recovery period
    flights = scenario_data["flights"]
    total_flights = len(flights)
    total_flight_minutes_in_recovery = 0
    total_flight_minutes_total = 0
    
    for flight_id, flight_data in flights.items():
        dep_time_str = flight_data["DepTime"]  
        arr_time_str = flight_data["ArrTime"] 
        
        dep_minutes = time_to_minutes(dep_time_str)
        arr_minutes = time_to_minutes(arr_time_str)
        
        total_flight_minutes_total += arr_minutes - dep_minutes
        overlap_start = max(dep_minutes, recovery_start_minutes)
        overlap_end = min(arr_minutes, recovery_end_minutes)
        
        if overlap_end > overlap_start:
            flight_overlap = overlap_end - overlap_start
        else:
            flight_overlap = 0
        
        total_flight_minutes_in_recovery += flight_overlap
    
    # Calculate total aircraft-minutes available during the recovery period
    total_recovery_aircraft_minutes = total_recovery_period_minutes * total_aircraft
    
    # Slack calculation
    if total_recovery_aircraft_minutes == 0:
        slack = 1.0
    else:
        slack = 1 - (total_flight_minutes_in_recovery / total_recovery_aircraft_minutes)
    
    return slack, total_flights, total_flight_minutes_total


def extract_disruption_stats(scenario_data):
    """
    Extract disruption statistics:
    - Count of fully disrupted (prob = 1.0)
    - Count of uncertain disruptions (0 < prob < 1.0)
    - Average probability across all aircraft (where an aircraft's probability is the max disruption probability it faces, 
      with 1.0 for fully disrupted and 0.0 if no disruption)
    - Average uncertainty probability (average of all disruptions where 0<prob<1.0, excluding 0 and 1)
    """
    disruptions_info = scenario_data.get('disruptions', {})
    disruptions_list = disruptions_info.get('disruptions', [])
    total_aircraft = disruptions_info.get('total_aircraft', 0)

    if total_aircraft == 0:
        # No aircraft or no disruptions
        return 0, 0, 0.0, 0.0

    fully_disrupted_count = sum(1 for d in disruptions_list if d.get('probability', 0.0) == 1.0)
    uncertain_disruptions = [d for d in disruptions_list if 0.0 < d.get('probability', 0.0) < 1.0]
    uncertain_count = len(uncertain_disruptions)

    aircraft_ids = scenario_data.get('aircraft_ids', [])
    ac_prob_map = {ac: 0.0 for ac in aircraft_ids}  
    
    for d in disruptions_list:
        ac_id = d.get('aircraft_id')
        p = d.get('probability', 0.0)
        # Keep the max probability for that aircraft
        if ac_id in ac_prob_map:
            ac_prob_map[ac_id] = max(ac_prob_map[ac_id], p)

    avg_ac_prob = sum(ac_prob_map.values()) / total_aircraft if total_aircraft > 0 else 0.0

    # Average uncertainty probability (only consider disruptions where 0<prob<1)
    if len(uncertain_disruptions) > 0:
        avg_uncertainty_prob = np.mean([d['probability'] for d in uncertain_disruptions])
    else:
        avg_uncertainty_prob = 0.0

    return fully_disrupted_count, uncertain_count, avg_ac_prob, avg_uncertainty_prob, total_aircraft

# Path to the scenarios folder
scenario_folder_path = "../../logs/scenarios/"
# latest_folder = max(
#     [f for f in os.listdir(scenario_folder_path) if f.startswith("scenario_folder_")],
#     key=lambda x: int(x.split('_')[-1].replace('.json', ''))
# )

# latest_folder = "scenario_folder_scenario_74.json" # Training/6ac-10-superdiverse

# latest_folder = "scenario_folder_scenario_77.json" # Training/6ac-10000-superdiverse
# latest_folder = "scenario_folder_scenario_4.json" # Testing/6ac-700-diverse
latest_folder = "scenario_folder_scenario_76.json" # Training/6ac-26-lilac

file_path = os.path.join(scenario_folder_path, latest_folder)

# Extract scenario ID
scenario_id = file_path.split('_')[-1].split('.')[0]
print(f"Scenario ID: {scenario_id}")

# Load the JSON data
with open(file_path, 'r') as file:
    data = json.load(file)

# Extract the scenarios from the JSON data
scenarios = data['outputs']


# Extract the data_folder (not strictly necessary for slack calculation, but we print it for context)
data_folder = data['data_folder']
print(f"Data Folder: {data_folder}")

# Calculate slack and disruption stats for each scenario and store in a list of dicts
results = []
for scenario_name, scenario_data in scenarios.items():
    scenario_slack, total_flights, total_flight_minutes_total = calculate_slack_for_scenario(scenario_data)
    fully_disrupted_count, uncertain_count, avg_ac_prob, avg_uncertain_prob, total_aircraft = extract_disruption_stats(scenario_data)
    results.append({
        "Scenario": scenario_name,
        "ScenarioSlack": scenario_slack,
        "TotalFlights": total_flights,
        "TotalFlightMinutes": total_flight_minutes_total,
        "FullyDisruptedCount": fully_disrupted_count,
        "UncertainCount": uncertain_count,
        "AvgAircraftProbability": avg_ac_prob,
        "AvgUncertaintyProbability": avg_uncertain_prob,
        "TotalAircraft": total_aircraft
    })

# Convert results to DataFrame
scenarios_df = pd.DataFrame(results)
print(scenarios_df)

# Save the slack results to CSV
# output_file = os.path.join(scenario_folder_path, f"scenario_slack_metrics_{scenario_id}.csv")
# scenarios_df.to_csv(output_file, index=False)
# print(f"Slack metrics saved to {output_file}")


Scenario ID: 76
Data Folder: ../data/TRAINING/6ac-26-lilac/
                        Scenario  ScenarioSlack  TotalFlights  \
0   deterministic_Scenario_00001       0.322037            19   
1   deterministic_Scenario_00002       0.375714            23   
2   deterministic_Scenario_00003       0.301068            17   
3   deterministic_Scenario_00004       0.329798            24   
4   deterministic_Scenario_00005       0.331650            20   
5   deterministic_Scenario_00006       0.309829            17   
6   deterministic_Scenario_00007       0.388725            20   
7   deterministic_Scenario_00008       0.286380            19   
8      stochastic_Scenario_00009       0.365185            18   
9      stochastic_Scenario_00010       0.253704            18   
10     stochastic_Scenario_00011       0.295833            21   
11     stochastic_Scenario_00012       0.344781            19   
12     stochastic_Scenario_00013       0.285522            21   
13     stochastic_Scenario_000

<div class="alert alert-block alert-success">
</br>
</br>
</br>
<b>DONE: </b>MERGED DATASET
</br>
</br>
</br>
</br>
</div>

In [4]:
import os
import pandas as pd

scenario_folder_path = "../../logs/inference_metrics/"
# unpack results_df
results_df = pd.read_csv(os.path.join(scenario_folder_path, f"6ac-26-lilac_{len(seeds)}_seeds.csv"))


# Merge scenario-level info from scenarios_df into results_df
merged_df = results_df.merge(scenarios_df, on='Scenario', how='left')

# Add scenario category based on prefix and difficulty level
merged_df["ScenarioCategory"] = merged_df["Scenario"].apply(
    lambda x: "Deterministic" if x.startswith("deterministic") else
             "Stochastic High" if x.startswith("stochastic_high") else
             "Stochastic Medium" if x.startswith("stochastic_medium") else
             "Stochastic Low" if x.startswith("stochastic_low") else
             "Mixed High" if x.startswith("mixed_high") else
             "Mixed Medium" if x.startswith("mixed_medium") else
             "Mixed Low" if x.startswith("mixed_low") else
             "Other"
)

# Sort models in desired order
model_order = ['proactive', 'myopic', 'reactive', 'greedy_reactive']
merged_df['Model_Type'] = merged_df['Model'].str.extract('(' + '|'.join(model_order) + ')')
merged_df = merged_df.sort_values('Model_Type')
merged_df["Model"] = merged_df["Model_Type"]
merged_df = merged_df.drop('Model_Type', axis=1)
merged_df_backup = merged_df.copy()


# Update model names in merged_df
merged_df['Model'] = merged_df['Model'].apply(lambda x: 
    'DQN Proactive-U' if x.startswith('proactive') else
    'DQN Proactive-N' if x.startswith('myopic') else 
    'DQN Reactive' if x.startswith('reactive') else
    'Greedy Reactive' if x.startswith('greedy_reactive') else
    x
)

print("Inference Results (After Merging):")
print(merged_df)

# Save the merged results to CSV
merged_output_file = os.path.join(scenario_folder_path, f"scenario_inference_metrics_{scenario_id}.csv")
merged_df.to_csv(merged_output_file, index=False)
print(f"Inference results with scenario info saved to {merged_output_file}")



# print all column names
print("==== Columns: ====")
print(merged_df.columns)

print("==== amount of rows: ====")
print(len(merged_df))

print("==== Models: ====")
print(merged_df["Model"].unique())

print('===== len(seeds) =====')
print(len(merged_df['Seed'].unique()))

print('===== len(scenarios) =====')
print(len(merged_df['Scenario'].unique()))


Inference Results (After Merging):
                         Scenario            Model  Seed  TotalReward  \
0    deterministic_Scenario_00001  DQN Proactive-N     1     -10126.0   
225  deterministic_Scenario_00008  DQN Proactive-N     1     -30007.0   
705     stochastic_Scenario_00018  DQN Proactive-N     1     -15396.4   
706     stochastic_Scenario_00018  DQN Proactive-N     2     -15396.4   
707     stochastic_Scenario_00018  DQN Proactive-N     3       -216.0   
..                            ...              ...   ...          ...   
374          mixed_Scenario_00025     DQN Reactive     5     -20398.0   
373          mixed_Scenario_00025     DQN Reactive     4     -15398.0   
372          mixed_Scenario_00025     DQN Reactive     3     -20398.0   
434     stochastic_Scenario_00009     DQN Reactive     5     -10331.6   
779     stochastic_Scenario_00020     DQN Reactive     5      23790.5   

     TotalDelays  TotalCancelledFlights  ScenarioTime  ScenarioSteps  \
0            0.0

<div class="alert alert-block alert-info">
<b>Check: </b>Comparison of Models Across All Scenarios
</div>

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

# Define model colors and order
model_colors = {
    'DQN Proactive-U': ('orange', 'DQN Proactive-U'),
    'DQN Proactive-N': ('blue', 'DQN Proactive-N'),
    'DQN Reactive': ('green', 'DQN Reactive'),
    'Greedy Reactive': ('darkgrey', 'Greedy Reactive')
}

# First aggregate by Model and Seed, then calculate mean and std across seeds
comparison_table = (
    merged_df
    .groupby(['Model', 'Seed'])
    .agg(
        TotalReward=('TotalReward', 'mean'),
        ScenarioTime=('ScenarioTime', 'mean'), 
        ScenarioSteps=('ScenarioSteps', 'mean'),
        TotalDelays=('TotalDelays', 'mean'),
        TotalCancelledFlights=('TotalCancelledFlights', 'mean'),
        TotalTailSwaps=('TailSwaps', 'mean')
    )
    .groupby('Model')
    .agg(
        Mean_Reward=('TotalReward', 'mean'),
        Std_Reward=('TotalReward', 'std'),
        Mean_Runtime=('ScenarioTime', 'mean'),
        Std_Runtime=('ScenarioTime', 'std'),
        Mean_Steps=('ScenarioSteps', 'mean'),
        Std_Steps=('ScenarioSteps', 'std'),
        Mean_Delays=('TotalDelays', 'mean'),
        Std_Delays=('TotalDelays', 'std'),
        Mean_CancelledFlights=('TotalCancelledFlights', 'mean'),
        Std_CancelledFlights=('TotalCancelledFlights', 'std'),
        Mean_TailSwaps=('TotalTailSwaps', 'mean'),
        Std_TailSwaps=('TotalTailSwaps', 'std')
    )
    .round(2)
)

# Sort the comparison table according to specified order
model_order = ['Greedy Reactive', 'DQN Reactive', 'DQN Proactive-N', 'DQN Proactive-U']
comparison_table = comparison_table.reindex(model_order)

print("Comparison of Models Across All Scenarios:")
print(comparison_table)


Comparison of Models Across All Scenarios:
                 Mean_Reward  Std_Reward  Mean_Runtime  Std_Runtime  \
Model                                                                 
Greedy Reactive          NaN         NaN           NaN          NaN   
DQN Reactive        -8521.58      370.64          0.04         0.00   
DQN Proactive-N     -7052.76      434.13          0.04         0.01   
DQN Proactive-U     -4395.31      836.57          0.04         0.00   

                 Mean_Steps  Std_Steps  Mean_Delays  Std_Delays  \
Model                                                             
Greedy Reactive         NaN        NaN          NaN         NaN   
DQN Reactive           7.77       0.03         6.22        1.50   
DQN Proactive-N        7.53       0.07       134.46        3.51   
DQN Proactive-U        7.25       0.10       169.26        0.74   

                 Mean_CancelledFlights  Std_CancelledFlights  Mean_TailSwaps  \
Model                                          

In [6]:
"""
info on merged_df:

[280000 rows x 27 columns]
Inference results with scenario info saved to ../logs/scenarios/scenario_inference_metrics_4.csv
==== Columns: ====
Index(['Scenario', 'Model', 'Seed', 'TotalReward', 'TotalDelays',
       'TotalCancelledFlights', 'ScenarioTime', 'ScenarioSteps',
       'ScenarioResolvedConflicts', 'SolutionSlack', 'TailSwaps',
       'ActualDisruptedFlights', 'Reward_delay_penalty_total',
       'Reward_cancel_penalty', 'Reward_inaction_penalty',
       'Reward_proactive_bonus', 'Reward_time_penalty',
       'Reward_final_conflict_resolution_reward', 'ScenarioSlack',
       'TotalFlights', 'TotalFlightMinutes', 'FullyDisruptedCount',
       'UncertainCount', 'AvgAircraftProbability', 'AvgUncertaintyProbability',
       'TotalAircraft', 'ScenarioCategory'],
      dtype='object')
==== amount of rows: ====
280000
==== Models: ====
['Greedy Reactive' 'DQN Proactive-N' 'DQN Proactive-U' 'DQN Reactive']
===== len(seeds) =====
100
===== len(scenarios) =====
700
"""
# Get scenarios starting with "mixed" for DQN Proactive-U model
mixed_scenarios = merged_df[
    (merged_df['Model'] == 'DQN Proactive-U') & 
    (merged_df['Scenario'].str.startswith('mixed'))
]

# Group by scenario and calculate mean reward
scenario_means = mixed_scenarios.groupby('Scenario').agg({
    'TotalReward': 'mean',
    'ActualDisruptedFlights': 'mean'
}).reset_index()

# Get top 5 scenarios by average reward
top_5_scenarios = scenario_means.nlargest(5, 'TotalReward')
print("\nTop 5 'mixed' scenarios for DQN Proactive-U by average reward:")
print(top_5_scenarios)



Top 5 'mixed' scenarios for DQN Proactive-U by average reward:
               Scenario  TotalReward  ActualDisruptedFlights
5  mixed_Scenario_00026     12253.95                     5.0
1  mixed_Scenario_00022     -1301.70                     3.0
3  mixed_Scenario_00024     -1354.88                     3.2
0  mixed_Scenario_00021     -9186.66                     2.5
4  mixed_Scenario_00025    -23350.90                     3.7
